Project 1 - California Water Usage
=================

Welcome to the first project in Data 8! We will be exploring possible connections between water usage, geography, and income in California. The water data for this project was procured from the [California State Water Resources Control Board](http://www2.pacinst.org/gpcd/table.html) and curated by the [Pacific Institute](http://pacinst.org/). The map data includes [US topography](https://github.com/jgoodall/us-maps), [California counties](https://github.com/johan/world.geo.json/tree/master/countries/USA/CA), and [ZIP codes](http://bl.ocks.org/jefffriesen/6892860).

The dataset on income comes from the IRS ([documentation](http://www.irs.gov/pub/irs-soi/13zpdoc.doc)).  We have identified some interesting columns in the dataset, but a full description of all the columns (and a definition of the population in the dataset and some interesting anonymization procedures they used) is available in this [description](irs_info.pdf).

As usual, **run the cell below** to prepare the automatic tests. **Passing the automatic tests does not guarantee full credit on any question.** The tests are provided to help catch some common errors, but it is *your* responsibility to answer the questions correctly.

In [38]:
# Run this cell, but please don't change it.

import numpy as np
import math
from datascience import *

# These lines set up the plotting functionality and formatting.
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# These lines load the tests.
from client.api.assignment import load_assignment 
project1 = load_assignment('project1.ok')

First, load the data. Loading may take some time.

In [39]:
# Run this cell, but please don't change it.

districts = Map.read_geojson('water_districts.geojson')
zips = Map.read_geojson('ca_zips.geojson.gz')
usage_raw = Table.read_table('water_usage.csv', dtype={'pwsid': str})
income_raw = Table.read_table('ca_income_by_zip.csv', dtype={'ZIP': str}).drop(['STATEFIPS', 'STATE', 'agi_stub'])
wd_vs_zip = Table.read_table('wd_vs_zip.csv', dtype={'PWSID': str, 'ZIP': str}).set_format([2, 3], PercentFormatter)

Part 0: Maps
======

The `districts` and `zips` data sets are `Map` objects. Documentation on mapping in the `datascience` package can be found at [data8.org/datascience/maps.html](http://data8.org/datascience/maps.html).  To view a map of California's water districts, run the cell below. Click on a district to see its description.

In [40]:
districts.format(width=400, height=200)

A `Map` is a collection of regions and other features such as points and markers, each of which has a **string** `id` and various properties. You can view the features of the `districts` map as a table using `Table.from_records`.

In [41]:
district_table = Table.from_records(districts.features)
district_table.show(3)

To display a `Map` containing only two features from the `district_table`, call `Map` on a list containing those two features from the `feature` column.
    
**Question 0.0** Draw a map of the Alameda County Water District (row 0) and the East Bay Municipal Utilities District (row 2).

In [42]:
alameda_and_east_bay = ...
Map(alameda_and_east_bay, height=300, width=300)

In [43]:
_ = project1.grade('q00')

*Hint*: If scrolling becomes slow on your computer, you can clear maps for the cells above by running `Cell > All Output > Clear` from the `Cell` menu. 

Part 1: California Income
=======

Let's look at the `income_raw` table.

In [44]:
income_raw

Some observations:

1. The table contains several numerical columns and a column for the ZIP code.
2. For each ZIP code, there are 6 rows.  Each row for a ZIP code has data from tax returns in one *income bracket* -- a group of people who make between some income and some other income.  
3. According to the IRS documentation, all the numerical columns are *totals* -- either total numbers of returns that fall into various categories, or total amounts of money (in thousands of dollars) from returns in those categories.  For example, the column `'N02650'` is the number of returns that included a total income amount, and `'A02650'` is the total amount of total income (in thousands of dollars) from those returns.

**Question 1.0.** Since we don't care about income brackets, but we do care about totals per ZIP code, let's group together our income data by ZIP code. Assign the name `income_by_zipcode` to a table with just one row per ZIP code. When you group according to ZIP code, the remaining columns should be summed. In other words, for any other column such as `'N02650'`, the value of `'N02650'` in a row corresponding to ZIP code 90210 (for example) should be the sum of the values of `'N02650'` in the 6 rows of `income_raw` corresponding to ZIP code 90210.

In [45]:
income_by_zipcode = ...
income_by_zipcode

In [46]:
_ = project1.grade('q10')

**Question 1.1.** Relabel the columns in `income_by_zipcode` to match the labels in `income_raw`; you probably modified all the names slightly in the previous question. 

*Hint:* Inspect `income_raw.labels` and `income_by_zipcode.labels` to find the differences you need to change.

*Hint 2:* Since there are many columns, it will be easier to relabel each of them by using a `for` statement. See [Section 3.2](http://www.inferentialthinking.com/chapter3/iteration.html)  of the textbook for details.

*Hint 3:* You can use the `replace` method of a string to remove excess content. See [lab02](https://github.com/data-8/data8assets/tree/gh-pages/labs/lab02/lab02.ipynb) for examples.

*Hint 4:* To create a new table from an existing table with one label replaced, use `relabeled`. To **change** a label in an existing table permanently, use `relabel`. Both methods take two arguments: the old label and the new label. You can solve this problem with either one, but `relabel` is simpler.

In [47]:
...
    ...
income_by_zipcode

In [48]:
_ = project1.grade('q11')

**Question 1.2.**
Create a table called `income` with  one row per ZIP code and the following columns.

1. A `ZIP` column with the same contents as `'ZIP'` from `income_by_zipcode`.
2. A `returns` column containing the total number of tax returns that include a total income amount (column `'N02650'` from `income_by_zipcode`).
3. A `total` column containing the total income in all tax returns in thousands of dollars (column `'A02650'` from `income_by_zipcode`).
4. A `farmers` column containing the number of farmer returns (column `'SCHF'` from `income_by_zipcode`).

In [49]:
income = Table().with_columns([
        ...
        ...
        ...
        ...
        ])
income.set_format('total', NumberFormatter(0)).show(5)

In [50]:
_ = project1.grade('q12')

**Question 1.3.** What is the average total income reported on all California tax returns that include a total income amount? **Express the answer in *dollars* as an `int` rounded to the nearest dollar.**

In [51]:
...

**Question 1.4.** All ZIP codes with less than 100 returns (or some other special conditions) are grouped together into one ZIP code with a special code. Remove the row for that ZIP code from the `income` table. *Hint:* This ZIP code value has far more returns than any of the other ZIP codes.

*Hint:* To **remove** a row in the `income` table using `where`, assign `income` to the smaller table using the following expression structure:

    income = income.where(...)
    
*Hint 2:* Each ZIP code is represented as a string, not an `int`.

In [52]:
income = ...

In [53]:
_ = project1.grade('q14')

**Question 1.5**. Among the tax returns in California for ZIP codes represented in the `incomes` table, is there an association between income and living in a ZIP code with a higher-than-average proportion of farmers?

Answer the question by comparing the average incomes for two groups of *tax returns*: those in ZIP codes with a greater-than-average proportion of farmers and those in ZIP codes with a less-than-average (or average) proportion. Make sure both of these values are displayed (preferably in a table). *Then, describe your findings.*

In [54]:
# Build and display a table with two rows:
#   1) incomes of returns in ZIP codes with a greater-than-average proportion of farmers
#   2) incomes of returns in other ZIP codes
farm_proportion = income.column('farmers')/income.column('returns')
income_by_farm_proportion = ...
...

*Replace this line with a description of your findings.*

**Question 1.6.** Investigate the same question by comparing two histograms: the average incomes of ZIP codes that have above-average vs below-average proportions of farmers. Quantify and describe the difference in the [standard deviations](www.inferentialthinking.com/chapter3/spread.html) of average incomes for the two kinds of ZIP codes.

<img src="https://i.imgur.com/jicA2to.png"/>

In [73]:
# You do not need to change this cell; just look at the chart it generates.

# Note: there was provided code here to draw the histogram above, but it
# did not work for everyone, so it has been commented out below.
# If you don't see a chart above, it's here: https://i.imgur.com/jicA2to.png

# bins = np.arange(20000, 300000, 5000)
# avg_income = 1000 * income.column('total')/income.column('returns')
# binned_incomes=Table()\
#     .with_column('income', avg_income)\
#     .where(farm_proportion > np.mean(farm_proportion))\
#     .bin(bins=bins).relabeled(1, 'Avg. income in ZIP codes with above-average farmer proportion')\
#     .join('bin', Table().with_column('income', avg_income)\
#         .where(farm_proportion <= np.mean(farm_proportion))\
#         .bin(bins=bins).relabeled(1, 'Avg. income of ZIP codes with below-average farmer proportion'))
# binned_incomes.hist()

In [56]:
# Compute and compare the spread of both distributions
...

*Replace this line with a description of your findings.*

ZIP codes cover all the land in California and do not overlap. Here's a map of all of them.

<img src="california-zip-code-map.jpg" alt="CA ZIP Codes"/>

**Question 1.7.** Among the ZIP codes represented in the `incomes` table, is there an association between high average income and some aspect of the ZIP code's location? If so, describe one aspect of the location that is clearly associated with high income.

Answer the question by drawing a map of all ZIP codes that have an average income above 100,000 dollars. *Then, describe an association that you observe.*

In order to create a map of certain ZIP codes, you need to 
- Construct a table containing only the ZIP codes of interest, called `high_average_zips`,
- Join `high_average_zips` with the `zip_features` table to find the region for each ZIP code of interest,
- Call `Map(...)` on the column of features (provided).

In [57]:
# Write code to draw a map of only the high-income ZIP codes
zip_features = Table.from_records(zips.features)
high_average_zips = ...
high_zips_with_region = ...
Map(list(high_zips_with_region.column('feature')), width=400, height=300)

*Replace this line with a description of your findings.*

In [58]:
_ = project1.grade('q17')

Part 2: Water Usage
======
We will now investigate water usage in California. The `usage` table contains three columns:

- `PWSID`: The Public Water Supply Identifier of the district
- `Population`: Estimate of average population served in 2015
- `Water`: Average residential water use (gallons per person per day) in 2014-2015

In [59]:
# Run this cell to create the usage table

usage_raw.set_format(4, NumberFormatter)
max_pop = usage_raw.select([0, 'population']).group(0, max).relabeled(1, 'Population')
avg_water = usage_raw.select([0, 'res_gpcd']).group(0, np.mean).relabeled(1, 'Water')
usage = max_pop.join('pwsid', avg_water).relabeled(0, 'PWSID')
usage

**Question 2.1.** Draw a map of the water districts, colored by the per capita water usage in each district.

Use the `districts.color(...)` method to generate the map. It takes as its first argument a two-column table with one row per district that has the district `PWSID` as its first column. The label of the second column is used in the legend of the map, and the values are used to color each region.

In [60]:
per_capita_usage = ...
districts.color(per_capita_usage, key_on='feature.properties.PWSID') 

In [61]:
_ = project1.grade('q21')

**Question 2.2.** Based on the map above, which part of California appears to use more water per person, the San Francisco area or the Los Angeles area? 

*Replace this line with a description of your findings.*

Next, we will try to match each ZIP code with a water district. ZIP code boundaries do not always line up with water districts, and one water district often covers multiple ZIP codes, so this process is imprecise. It is even the case that some water districts overlap each other. Nonetheless, we can continue our analysis by matching each ZIP code to the water district with the largest geographic overlap.

The table `wd_vs_zip` describes the proportion of land in each ZIP code that is contained in each water district and vis versa. (The proportions are approximate because they do not correctly account for discontiguous districts, but they're mostly accurate.)

In [62]:
wd_vs_zip.show(5)

**Question 2.3.** Complete the `district_for_zip` function that takes a ZIP code. It returns the PWSID with the largest value of `ZIP in District` for that `zip_code`, if that value is at least 50%. Otherwise, it returns the string `'No District'`.

In [63]:
def district_for_zip(zip_code):
    zip_code = str(zip_code) # Ensure that the ZIP code is a string, not an integer
    districts = ...
    at_least_half = ...
    if at_least_half:
        ...
    else:
        return 'No District'

district_for_zip(94709)

In [64]:
_ = project1.grade('q23')

This function can be used to associate each ZIP code in the `income` table with a `PWSID` and discard ZIP codes that do not lie (mostly) in a water district.

In [65]:
zip_pwsids = income.apply(district_for_zip, 'ZIP')
income_with_pwsid = income.with_column('PWSID', zip_pwsids).where(zip_pwsids != "No District")
income_with_pwsid.set_format(2, NumberFormatter(0)).show(5)

**Question 2.4.** Create a table called `district_data` with one row per PWSID and the following columns:

- `PWSID`: The ID of the district
- `Population`: Population estimate
- `Water`: Average residential water use (gallons per person per day) in 2014-2015
- `Income`: Average income in dollars of all tax returns in ZIP codes that are (mostly) contained in the district according to `income_with_pwsid`.

*Hint*: First create a `district_income` table that sums the incomes and returns for ZIP codes in each water district.

In [66]:
district_income = ...
district_data = ...
district_data.set_format(['Population', 'Water', 'Income'], NumberFormatter(0))

In [67]:
_ = project1.grade('q24')

**Question 2.5.** The `bay_districts` table gives the names of all water districts in the San Francisco Bay Area. Is there an association between water usage and income among Bay Area water districts? Use the tables you have created to compare water usage between the 10 Bay Area water districts with the highest average income and the rest of the Bay Area districts, then describe the association. *Do not include any districts in your analysis for which you do not have income information.*

The names below are just suggestions; you may perform the analysis in any way you wish.

*Note*: Some Bay Area water districts may not appear in your `district_data` table. That's ok. Perform your analysis only on the subset of districts where you have both water usage & income information.

In [72]:
bay_districts = Table.read_table('bay_districts.csv')
bay_water_vs_income = ...
top_10 = ...
...

*Complete this one-sentence conclusion:* In the Bay Area, people in the top 10 highest-income water districts used an average of `_________` more gallons of water per person per day than people in the rest of the districts.

**Question 2.6.** In one paragraph, summarize what you have discovered through the analyses in this project and suggest what analysis should be conducted next to better understand California water usage, income, and geography. What additional data would be helpful in performing this next analysis?

*Replace this line with your conclusion paragraph.*

Congratulations - you've finished Project 1 of Data 8! 

To submit:

1. Select `Run All` from the `Cell` menu to ensure that you have executed all cells, including the test cells.  Make sure that the visualizations you create are actually displayed.
2. Select `Download as PDF via LaTeX (.pdf)` from the `File` menu.  (Sometimes that seems to fail.  If it does, you can download as HTML, open the .html file in your browser, and print it to a PDF.)
3. Read that file! If any of your lines are too long and get cut off, we won't be able to see them, so break them up into multiple lines and download again. If maps do not appear in the output, that's ok.
4. Submit that downloaded file (called `project1.pdf`) to Gradescope.

If you cannot submit online, come to office hours for assistance. The office hours
schedule appears on [data8.org/weekly](http://data8.org/weekly).

In [69]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [project1.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]

If you want, draw some more maps below.

In [70]:
# Your extensions here (completely optional)